In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import re

In [ ]:
df = pd.read_csv('./Twitter_Data.csv')
df.head(5)

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [ ]:
df['category'] = df['category'].replace({1.0: 'Positive', 0.0: 'Neutral', -1.0: 'Negative'})
data = df.rename(columns={'clean_text': 'text', 'category': 'sentiment'})
print(data)

                                                     text sentiment
0       when modi promised “minimum government maximum...  Negative
1       talk all the nonsense and continue all the dra...   Neutral
2       what did just say vote for modi  welcome bjp t...  Positive
3       asking his supporters prefix chowkidar their n...  Positive
4       answer who among these the most powerful world...  Positive
...                                                   ...       ...
162975  why these 456 crores paid neerav modi not reco...  Negative
162976  dear rss terrorist payal gawar what about modi...  Negative
162977  did you cover her interaction forum where she ...   Neutral
162978  there big project came into india modi dream p...   Neutral
162979  have you ever listen about like gurukul where ...  Positive

[162980 rows x 2 columns]


In [ ]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].fillna('').astype(str)
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

<ipython-input-5-8af2e4d53762>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].fillna('').astype(str)
<ipython-input-5-8af2e4d53762>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())


144500
71020


In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 48, 128)           256000    
                                                                 
 spatial_dropout1d (Spatial  (None, 48, 128)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511194 (1.95 MB)
Trainable params: 511194 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


Hereby I declare the train and test dataset.

In [ ]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(72203, 48) (72203, 2)
(35564, 48) (35564, 2)


Here we train the Network. We should run much more than 7 epoch, but I would have to wait forever for kaggle, so it is 7 for now.

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
2257/2257 - 534s - loss: 0.3045 - accuracy: 0.8685 - 534s/epoch - 237ms/step
Epoch 2/7
2257/2257 - 515s - loss: 0.2050 - accuracy: 0.9202 - 515s/epoch - 228ms/step
Epoch 3/7
2257/2257 - 507s - loss: 0.1906 - accuracy: 0.9280 - 507s/epoch - 224ms/step
Epoch 4/7
2257/2257 - 500s - loss: 0.1785 - accuracy: 0.9333 - 500s/epoch - 222ms/step
Epoch 5/7
2257/2257 - 497s - loss: 0.1686 - accuracy: 0.9380 - 497s/epoch - 220ms/step
Epoch 6/7
2257/2257 - 491s - loss: 0.1589 - accuracy: 0.9404 - 491s/epoch - 218ms/step
Epoch 7/7
2257/2257 - 487s - loss: 0.1493 - accuracy: 0.9444 - 487s/epoch - 216ms/step


Extracting a validation set, and measuring score and accuracy.

In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

1065/1065 - 15s - loss: 0.2046 - accuracy: 0.9265 - 15s/epoch - 14ms/step
score: 0.20
acc: 0.93


In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):

    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]

    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1

    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 0s - 240ms/epoch - 240ms/step
1/1 - 0s - 28ms/epoch - 28ms/step
1/1 - 0s - 26ms/epoch - 26ms/step
1/1 - 0s - 25ms/epoch - 25ms/step
1/1 - 0s - 29ms/epoch - 29ms/step
1/1 - 0s - 26ms/epoch - 26ms/step
1/1 - 0s - 25ms/epoch - 25ms/step
1/1 - 0s - 25ms/epoch - 25ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 37ms/epoch - 37ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 37ms/epoch - 37ms/step
1/1 - 0s - 37ms/epoch - 37ms/step
1/1 - 0s - 38ms/epoch - 38ms/step
1/1 - 0s - 37ms/epoch - 37ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 36ms/epoch - 36ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 38ms/epoch - 38ms/step
1/1 - 0s - 38ms/epoch - 38ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 38ms/epoch - 38ms/step
1/1 - 0s - 38ms/epoch - 38ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 3

In [ ]:
twt = ['Will I be successful?']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0   11 1798  441]]
1/1 - 0s - 22ms/epoch - 22ms/step
positive
